In [ ]:
# *********
# Environment & Settings
# *********

import os
from dotenv import load_dotenv

load_dotenv()

if not os.getenv("OPENAI_API_KEY"):
    raise RuntimeError("❌ OPENAI_API_KEY missing")

# --------- GLOBAL CONFIG ----------
EMBEDDING_DIM = 1536
SIMILARITY_THRESHOLD = 0.30
MAX_CACHE_SIZE = 100   # LRU limit
DB_PATH = "agent_memory.db"

print("✅ Environment & config loaded")


✅ Environment & config loaded


In [2]:
# ********
# LLM Initialization
# ********

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.2
)

print("✅ LLM initialized")


c:\Users\heman\Desktop\Agentic AI Projects\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ LLM initialized


In [3]:
# *********
# Embeddings
# *********

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small"
)

print("✅ Embeddings ready")


✅ Embeddings ready


In [4]:
# ********
# SQLite (FACT MEMORY + SESSION STORAGE)
# ********

import sqlite3
import json

conn = sqlite3.connect(DB_PATH, check_same_thread=False)
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS facts (
    session_id TEXT PRIMARY KEY,
    data TEXT
)
""")

conn.commit()

def load_facts(session_id: str) -> dict:
    cursor.execute("SELECT data FROM facts WHERE session_id=?", (session_id,))
    row = cursor.fetchone()
    return json.loads(row[0]) if row else {}

def save_facts(session_id: str, facts: dict):
    cursor.execute(
        "REPLACE INTO facts (session_id, data) VALUES (?, ?)",
        (session_id, json.dumps(facts))
    )
    conn.commit()

print("✅ Per-session fact memory ready")


✅ Per-session fact memory ready


In [5]:
# *******
# FAISS + LRU Cache
# *******

import faiss
import numpy as np
from collections import OrderedDict
from langchain_core.messages import AIMessage

index = faiss.IndexFlatL2(EMBEDDING_DIM)

# LRU cache: {vector_index: AIMessage}
CACHE = OrderedDict()

def embed(text: str) -> np.ndarray:
    return np.array(embeddings.embed_query(text), dtype="float32")


In [6]:
# *********
# Fact Extraction Prompt
# *********

from langchain_core.messages import SystemMessage

FACT_EXTRACT_PROMPT = SystemMessage(
    content="""
Extract factual information from user input.

Return ONLY valid JSON.
If none, return {}.

Fields:
- name
- likes
- dislikes
- profession
- location
"""
)


In [7]:
# ********
# Fact Extraction Function
# ********

from langchain_core.messages import HumanMessage

def extract_facts(user_text: str, session_id: str):
    facts = load_facts(session_id)

    response = llm.invoke([
        FACT_EXTRACT_PROMPT,
        HumanMessage(content=user_text)
    ])

    try:
        new_facts = json.loads(response.content)
        if isinstance(new_facts, dict):
            facts.update(new_facts)
            save_facts(session_id, facts)
    except Exception:
        pass


In [8]:
# ********
# Fact QA (STRICT)
# ********

FACT_QA_PROMPT = SystemMessage(
    content="""
Answer ONLY from provided facts.
If missing, respond: NOT_FOUND
"""
)

def answer_from_facts(user_text: str, session_id: str):
    facts = load_facts(session_id)
    if not facts:
        return None

    response = llm.invoke([
        FACT_QA_PROMPT,
        HumanMessage(content=f"Facts: {facts}\nQuestion: {user_text}")
    ])

    if response.content.strip() == "NOT_FOUND":
        return None

    return AIMessage(content=response.content)


In [9]:
# **********
# Confidence Check (ANTI-HALLUCINATION)
# **********

CONFIDENCE_PROMPT = SystemMessage(
    content="""
Is the following answer factual and confident?
Reply ONLY YES or NO.
"""
)

def is_confident(answer: str) -> bool:
    response = llm.invoke([
        CONFIDENCE_PROMPT,
        HumanMessage(content=answer)
    ])
    return response.content.strip().upper() == "YES"


In [10]:
# *****
# Agent State
# *****

from typing import TypedDict, Annotated, List
from langgraph.graph.message import add_messages

class AgentState(TypedDict):
    query: Annotated[List, add_messages]


In [11]:
# **********
# MAIN AGENT LOGIC (Production Core)
# **********

import time
from langchain_core.messages import HumanMessage

def chatbot(state: AgentState, session_id: str) -> AgentState:
    last_msg = state["query"][-1]

    if not isinstance(last_msg, HumanMessage):
        return state

    user_text = last_msg.content
    start = time.time()

    # 1️⃣ FACT MEMORY
    fact_answer = answer_from_facts(user_text, session_id)
    if fact_answer:
        print(f"⚡ FACT HIT ({(time.time()-start)*1000:.1f} ms)")
        state["query"].append(fact_answer)
        return state

    # 2️⃣ SEMANTIC CACHE
    vec = embed(user_text).reshape(1, -1)
    if index.ntotal > 0:
        dist, idx = index.search(vec, 1)
        if dist[0][0] < SIMILARITY_THRESHOLD:
            key = idx[0][0]
            if key in CACHE:
                CACHE.move_to_end(key)
                print("⚡ CACHE HIT")
                state["query"].append(CACHE[key])
                return state

    # 3️⃣ LLM
    print("🐢 LLM reasoning...")
    response = llm.invoke(state["query"])

    # 4️⃣ STORE (SAFE)
    extract_facts(user_text, session_id)

    if is_confident(response.content):
        index.add(vec)
        CACHE[index.ntotal - 1] = response
        if len(CACHE) > MAX_CACHE_SIZE:
            CACHE.popitem(last=False)

    state["query"].append(response)
    return state


In [12]:
# *******
# Session ID
# *******

import uuid

SESSION_ID = str(uuid.uuid4())
# print("🆕 Session:", SESSION_ID)


In [ ]:
# ******
# LangGraph Build
# ******

from langgraph.graph import StateGraph, START, END

graph = StateGraph(AgentState)

graph.add_node("chatbot", lambda s: chatbot(s, SESSION_ID))
graph.add_edge(START, "chatbot")
graph.add_edge("chatbot", END)

workflow = graph.compile()

print("✅ Production graph built")


✅ Production graph built


In [ ]:
# *******
# Chat Interface
# *******

from langchain_core.messages import AIMessage

def chat(text: str):
    result = workflow.invoke(
        {"query": [HumanMessage(content=text)]}
    )

    for msg in result["query"][-2:]:
        role = "User" if isinstance(msg, HumanMessage) else "AI"
        print(f"{role}: {msg.content}")

    print("-" * 60)


In [15]:
chat("My name is Hemant and I like cricket")

🐢 LLM reasoning...
User: My name is Hemant and I like cricket
AI: That's great, Hemant! Cricket is a popular sport enjoyed by many people around the world. Do you have a favorite team or player?
------------------------------------------------------------


In [16]:
chat("What is my name?")

⚡ FACT HIT (712.4 ms)
User: What is my name?
AI: Your name is Hemant.
------------------------------------------------------------


In [18]:
chat("What do I like?")

⚡ FACT HIT (912.2 ms)
User: What do I like?
AI: You like cricket.
------------------------------------------------------------


In [19]:
chat("What is LangGraph?")

🐢 LLM reasoning...
User: What is LangGraph?
AI: LangGraph is a graph-based programming language that allows users to create and manipulate graphs using a simple and intuitive syntax. It is designed to make working with graphs easier and more efficient, particularly for tasks such as data analysis, network modeling, and algorithm development. LangGraph provides a range of built-in functions and libraries for working with graphs, as well as the ability to define custom functions and algorithms.
------------------------------------------------------------
